In [1]:
!pip install mrtparse
!pip install networkx
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.14.0-py3-none-any.whl size=265476 sha256=a4256a1ce57f9da6d608d3414f56d59cc686221dfb8423ca376a609545955573
  Stored in directory: /root/.cache/pip/wheels/53/e0/d7/603097e3fed62f821523433801c09e04cd7a7610c7565bd5a3
Successfully built neo4j


In [2]:
import json
import sys
from mrtparse import *
import networkx as nx
import matplotlib.pyplot as plt

import os
import requests
import gzip
import shutil

from neo4j import GraphDatabase

In [3]:
# Define a function to create the graph in Neo4j
def create_graph(tx, from_node_id, to_node_id):
    query = (
        "MERGE (fromNode:Node {id: $from_node_id}) "
        "MERGE (toNode:Node {id: $to_node_id}) "
        "MERGE (fromNode)-[:CONNECTS]->(toNode)"
    )
    tx.run(query, from_node_id=from_node_id, to_node_id=to_node_id)

In [4]:
# Connect to the Neo4j database
uri = "neo4j+s://28e5b287.databases.neo4j.io"  # Change to match your Neo4j server settings
username = "neo4j"      # Change to your Neo4j username
password = "z9J3DPCCxGYELn99XDdFbFIBnWOwR5fdn4MiG_Nvdck"      # Change to your Neo4j password

# Create a Neo4j driver
driver = GraphDatabase.driver(uri, auth=(username, password))

In [8]:
server_url = "https://data.ris.ripe.net"
rrc_name = "rrc00"
year = "2017"
month = "09"
date = "01"
time = "0800"

zip_file_name = "bview.{}{}{}.{}.gz".format(year,month,date,time)
filename = "bview.{}{}{}.{}".format(year,month,date,time)

remote_url = server_url + "/" + rrc_name + "/" + year + "." + month + "/" + zip_file_name
print(remote_url)

https://data.ris.ripe.net/rrc00/2017.09/bview.20170901.0800.gz


In [12]:
r = requests.get(remote_url, allow_redirects=True)

if(r.status_code==200):

    ## Removing the zip file if it exists
    if os.path.isfile(zip_file_name):
        os.remove(zip_file_name)

    ## Removing content file if it exists
    if os.path.isfile(filename):
        os.remove(filename)

    ## Obtaining the zip file
    with open(zip_file_name, 'wb') as f:
        f.write(r.content)

    ## Obtaining the content file
    with gzip.open(zip_file_name, 'rb') as f_in:
        with open(filename, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    ## Removing the zip file after extraction
    if os.path.isfile(zip_file_name):
        os.remove(zip_file_name)
else:
    print('File does not exist')

In [13]:
check_list = []
check_json = []

with driver.session() as session:
    i = 0
    for entry in Reader(filename):

        curr_json = entry.data
        #check_json.append(curr_json)

        if list(curr_json['subtype'].values())[0]== "RIB_IPV4_UNICAST":

            if 'rib_entries' in curr_json.keys():

                entry_count = curr_json.get('entry_count',0)
                for j in range(entry_count):
                    curr_list = curr_json['rib_entries'][j]['path_attributes'][1]['value'][0]['value']
                    #check_list.append(curr_list)

                    N = len(curr_list)
                    for k in range(N-1):
                      if(curr_list[k+1]!=curr_list[k]): ## Avoiding same node cycles
                        session.write_transaction(create_graph, curr_list[k+1], curr_list[k])
            else:
                curr_list = curr_json['path_attributes'][1]['value'][0]['value']
                #check_list.append(curr_list)

                N = len(curr_list)
                for k in range(N-1):
                  if(curr_list[k+1]!=curr_list[k]):
                    session.write_transaction(create_graph, curr_list[k+1], curr_list[k])


            i += 1

        if(i==20):
            break

## Removing content file after reading
if os.path.isfile(filename):
    os.remove(filename)

<ipython-input-13-3d5ee3854cc2>:23: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_graph, curr_list[k+1], curr_list[k])
